[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/tulasiram58827/TTS_TFLite/blob/main/End_to_End_TTS.ipynb)

This notebook provides an end-to-end Text to Speech with a choice to choose both TTS Model and Vocoder

## About TTS Architecture

Speech Synthesis generally also called as Text-To-Speech (TTS). Synthesizing a speech from text generally consists of two steps.
Spectrograms are one of the most commonly used features to represent speech.
- `TTS model` : TTS Model generates mel spectrograms (don't worry about mel) from text.
- `Vocoder` : Vocoder generates an audio from spectrograms. There are different types of vocoders.
    - Algorithmic based
        - Griffin-Lim
    - Neural Network based
        - MelGAN
        - MB-MelGAN
        - Parallel WaveGAN

## Setup

In [2]:
!sudo apt-get install espeak
!pip install -q phonemizer

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'sudo apt autoremove' to remove it.
The following additional packages will be installed:
  espeak-data libespeak1 libportaudio2 libsonic0
The following NEW packages will be installed:
  espeak espeak-data libespeak1 libportaudio2 libsonic0
0 upgraded, 5 newly installed, 0 to remove and 49 not upgraded.
Need to get 1,219 kB of archives.
After this operation, 3,031 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libportaudio2 amd64 19.6.0-1 [64.6 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/main amd64 libsonic0 amd64 0.2.0-6 [13.4 kB]
Get:3 http://archive.ubuntu.com/ubuntu bionic/universe amd64 espeak-data amd64 1.48.04+dfsg-5 [934 kB]
Get:4 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libespeak1 amd64 1.48.04+dfsg-5 [145 

In [3]:
!git clone https://github.com/mozilla/TTS

%cd TTS
!git checkout c7296b3
!pip -q install -r requirements.txt
!python setup.py install
%cd ..

Cloning into 'TTS'...
remote: Enumerating objects: 11984, done.
remote: Total 11984 (delta 0), reused 0 (delta 0), pack-reused 11984
Receiving objects: 100% (11984/11984), 122.87 MiB | 28.98 MiB/s, done.
Resolving deltas: 100% (8385/8385), done.
/content/TTS
Note: checking out 'c7296b3'.

You are in 'detached HEAD' state. You can look around, make experimental
changes and commit them, and you can discard any commits you make in this
state without impacting any branches by performing another checkout.

If you want to create a new branch to retain commits you create, you may
do so (now or later) by using -b with the checkout command again. Example:

  git checkout -b <new-branch-name>

HEAD is now at c7296b3 add module requirement
     |████████████████████████████████| 235 kB 5.0 MB/s 
     |████████████████████████████████| 125 kB 59.4 MB/s 
     |████████████████████████████████| 32.4 MB 31.7 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages 

In [4]:
!git clone https://github.com/TensorSpeech/TensorFlowTTS.git
!cd TensorFlowTTS
!pip -q install /content/TensorFlowTTS/

Cloning into 'TensorFlowTTS'...
remote: Enumerating objects: 10679, done.
remote: Total 10679 (delta 0), reused 0 (delta 0), pack-reused 10679
Receiving objects: 100% (10679/10679), 133.52 MiB | 30.66 MiB/s, done.
Resolving deltas: 100% (5163/5163), done.
  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.
     |████████████████████████████████| 489.6 MB 23 kB/s 
     |████████████████████████████████| 1.1 MB 64.0 MB/s 
     |████████████████████████████████| 212 kB 64.3 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 1.3 MB 5

In [5]:
# Otherwise, Fastspeech2 will fail
!pip install -q tf-nightly

     |████████████████████████████████| 577.7 MB 16 kB/s 
     |████████████████████████████████| 1.7 MB 51.1 MB/s 
     |████████████████████████████████| 5.8 MB 38.0 MB/s 
     |████████████████████████████████| 438 kB 71.5 MB/s 


## Imports

In [6]:
import os
import torch
import time
import IPython

import sys
sys.path.append('/content/TensorFlowTTS')

import numpy as np
import tensorflow as tf
print(tf.__version__)

from TTS.tf.utils.tflite import load_tflite_model
from TTS.tf.utils.io import load_checkpoint
from TTS.utils.io import load_config
from TTS.utils.text.symbols import symbols, phonemes
from TTS.utils.audio import AudioProcessor
from TTS.utils.synthesis import synthesis

from tensorflow_tts.processor import LJSpeechProcessor
from tensorflow_tts.processor.ljspeech import valid_symbols
from tensorflow_tts.configs import FastSpeechConfig, FastSpeech2Config
from tensorflow_tts.configs import MelGANGeneratorConfig
from tensorflow_tts.inference import TFAutoModel, AutoConfig, AutoProcessor

from tensorflow_tts.models import TFFastSpeech, TFFastSpeech2
from tensorflow_tts.models import TFMelGANGenerator

2.10.0-dev20220720


/usr/local/lib/python3.7/dist-packages/resampy/interpn.py:114: NumbaWarning: The TBB threading layer requires TBB version 2019.5 or later i.e., TBB_INTERFACE_VERSION >= 11005. Found TBB_INTERFACE_VERSION = 9107. The TBB threading layer is disabled.
  _resample_loop_p(x, t_out, interp_win, interp_delta, num_table, scale, y)
/usr/local/lib/python3.7/dist-packages/tensorflow_addons/utils/ensure_tf_install.py:43: UserWarning: You are currently using a nightly version of TensorFlow (2.10.0-dev20220720). 
TensorFlow Addons offers no support for the nightly versions of TensorFlow. Some things might work, some other might not. 
If you encounter a bug, do not file an issue on GitHub.
  UserWarning,
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package cmudict to /root/nltk_data...
[nltk_data]   Unzipping corpora/cmudict.zip.


## Acknowledgments

- `Tacotron2` - TFLite Model provided by [Mozilla TTS Repository](https://github/mozilla/TTS/) and you can refer to this [Notebook](https://colab.research.google.com/github/mozilla/TTS/blob/master/notebooks/DDC_TTS_and_MultiBand_MelGAN_TFLite_Example.ipynb#scrollTo=4dnpE0-kvTsu) for creation of TFLite models.


- `FastSpeech2` - TFLite Model provided by [TensorFlow TTS](https://github.com/TensorSpeech/TensorFlowTTS/) and you can use this [Notebook](https://github.com/TensorSpeech/TensorFlowTTS/blob/master/notebooks/TensorFlowTTS_FastSpeech_with_TFLite.ipynb) for creation of TFLite Models.


- `MelGAN` - Pre-trained Model provided by [TensorFlow TTS Repository](https://github.com/TensorSpeech/TensorFlowTTS/tree/master/examples/melgan#pretrained-models-and-audio-samples) and TFLite Model created with this [Notebook](https://github.com/tulasiram58827/TTS_TFLite/blob/main/MelGAN_TFLite.ipynb).


- `MB-MelGAN` - TFLite Model provided by [Mozilla TTS Repository](https://github/mozilla/TTS/).


- `Parallel WaveGAN` - PyTorch weights are provided by [Parallel WaveGAN Repository](https://github.com/kan-bayashi/ParallelWaveGAN#results). We converted to TFLite. For the process of TFLite conversion you can view this [Notebook](https://github.com/tulasiram58827/TTS_TFLite/blob/main/Parallel_WaveGAN_TFLite.ipynb).

**Note** that these models are trained on `LJSpeech` dataset.

In [7]:
# Downloading Tacotron2 TFLite Model and its config
!gdown --id 17PYXCmTe0el_SLTwznrt3vOArNGMGo5v -O tts_model.tflite
!gdown --id 18CQ6G6tBEOfvCHlPqP8EBI4xWbrr9dBc -O config.json

# Downloading MB-MelGAN Vocoder TFLite Model and its config
!gdown --id 1aXveT-NjOM1mUr6tM4JfWjshq67GvVIO -O vocoder_model.tflite
!gdown --id 1Rd0R_nRCrbjEdpOwq6XwZAktvugiBvmu -O config_vocoder.json
!gdown --id 11oY3Tv0kQtxK_JPgxrfesa99maVXHNxU -O scale_stats.npy

# Downloading Parallel WaveGAN TFLite Models
!wget -q https://github.com/tulasiram58827/TTS_TFLite/raw/main/models/parallel_wavegan_dr.tflite
!wget -q https://github.com/tulasiram58827/TTS_TFLite/raw/main/models/parallel_wavegan_float16.tflite
    
# Downloading MelGAN TFlite Models
!wget -q https://github.com/tulasiram58827/TTS_TFLite/raw/main/models/melgan.tflite
!wget -q https://github.com/tulasiram58827/TTS_TFLite/raw/main/models/melgan_float16.tflite

# Downloading the Fastspeech2 model (available in dynamic-range quantization currently)
!wget -q https://github.com/tulasiram58827/TTS_TFLite/raw/main/models/fastspeech_quant.tflite

# Downloading ljspeech_mapper
!gdown --id {"1YBaDdMlhTXxsKrH7mZwDu-2aODq5fr5e"} -O ljspeech_mapper.json

/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=17PYXCmTe0el_SLTwznrt3vOArNGMGo5v
To: /content/tts_model.tflite
100% 30.1M/30.1M [00:00<00:00, 174MB/s] 
/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=18CQ6G6tBEOfvCHlPqP8EBI4xWbrr9dBc
To: /content/config.json
100% 9.53k/9.53k [00:00<00:00, 14.0MB/s]
/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From

## MelGAN Inference

In [8]:
def run_melgan(mel_spec, quantization):
    model_name = f'melgan_{quantization}.tflite'
    
    feats = np.expand_dims(mel_spec, 0)
    interpreter = tf.lite.Interpreter(model_path=model_name)
    
    interpreter = tf.lite.Interpreter(model_path=model_name)

    input_details = interpreter.get_input_details()

    output_details = interpreter.get_output_details()

    interpreter.resize_tensor_input(input_details[0]['index'],  [1, feats.shape[1], feats.shape[2]], strict=True)
    interpreter.allocate_tensors()

    interpreter.set_tensor(input_details[0]['index'], feats)

    interpreter.invoke()

    output = interpreter.get_tensor(output_details[0]['index'])
    
    return output

## MB MelGAN Inference

In [9]:
def run_mb_melgan(mel_spec):
  VOCODER_MODEL = "vocoder_model.tflite"
  VOCODER_CONFIG = "config_vocoder.json"
  vocoder_model = load_tflite_model(VOCODER_MODEL)  
  VOCODER_CONFIG = load_config(VOCODER_CONFIG)
  vocoder_inputs = mel_spec[None, :, :]
  # get input and output details
  input_details = vocoder_model.get_input_details()
  # reshape input tensor for the new input shape
  vocoder_model.resize_tensor_input(input_details[0]['index'], vocoder_inputs.shape)
  vocoder_model.allocate_tensors()
  detail = input_details[0]
  vocoder_model.set_tensor(detail['index'], vocoder_inputs)
  # run the model
  vocoder_model.invoke()
  # collect outputs
  output_details = vocoder_model.get_output_details()
  waveform = vocoder_model.get_tensor(output_details[0]['index'])
  return waveform 

## Parallel WaveGAN Inference

In [10]:
def run_parallel_wavegan(melspec, quantization):
    model_name = f'parallel_wavegan_{quantization}.tflite'
    feats = np.expand_dims(melspec, 0)
    interpreter = tf.lite.Interpreter(model_path=model_name)

    input_details = interpreter.get_input_details()

    output_details = interpreter.get_output_details()

    interpreter.resize_tensor_input(input_details[0]['index'],  [1, feats.shape[1], feats.shape[2]], strict=True)
    interpreter.allocate_tensors()

    interpreter.set_tensor(input_details[0]['index'], feats)

    interpreter.invoke()

    output = interpreter.get_tensor(output_details[0]['index'])
    
    return output

## FastSpeech Inference

Below inference code is copied from [Tensorflow TTS Notebook](https://github.com/TensorSpeech/TensorFlowTTS/blob/master/notebooks/TensorFlowTTS_FastSpeech_with_TFLite.ipynb)

In [11]:
# Prepare input data.
def fastspeech_prepare_input(input_ids):
  input_ids = tf.expand_dims(tf.convert_to_tensor(input_ids, dtype=tf.int32), 0)
  return (input_ids,
          tf.convert_to_tensor([0], tf.int32),
          tf.convert_to_tensor([1.0], dtype=tf.float32),
          tf.convert_to_tensor([1.0], dtype=tf.float32),
          tf.convert_to_tensor([1.0], dtype=tf.float32))

# Test the model on random input data.
def fastspeech_infer(tflite_model_path, input_text):
  # Load the TFLite model and allocate tensors.
  interpreter = tf.lite.Interpreter(model_path=tflite_model_path)

  # Get input and output tensors.
  input_details = interpreter.get_input_details()
  output_details = interpreter.get_output_details()

  processor = AutoProcessor.from_pretrained(pretrained_path="ljspeech_mapper.json")
  input_ids = processor.text_to_sequence(input_text.lower())
  interpreter.resize_tensor_input(input_details[0]['index'], 
                                  [1, len(input_ids)])
  interpreter.resize_tensor_input(input_details[1]['index'], 
                                  [1])
  interpreter.resize_tensor_input(input_details[2]['index'], 
                                  [1])
  interpreter.resize_tensor_input(input_details[3]['index'], 
                                  [1])
  interpreter.resize_tensor_input(input_details[4]['index'], 
                                  [1])
  interpreter.allocate_tensors()
  input_data = fastspeech_prepare_input(input_ids)
  for i, detail in enumerate(input_details):
    input_shape = detail['shape_signature']
    interpreter.set_tensor(detail['index'], input_data[i])

  interpreter.invoke()

  # The function `get_tensor()` returns a copy of the tensor data.
  # Use `tensor()` in order to get a pointer to the tensor.
  return (interpreter.get_tensor(output_details[0]['index']),
          interpreter.get_tensor(output_details[1]['index']))

## Tacotron2 Inference

In [12]:
def run_tacotron2(text):
    use_cuda = False
    TTS_MODEL = "tts_model.tflite"
    TTS_CONFIG = "config.json"
    TTS_CONFIG = load_config(TTS_CONFIG)
    ap = AudioProcessor(**TTS_CONFIG.audio)
    speaker_id = None
    speakers = []
    model = load_tflite_model(TTS_MODEL)
    waveform, alignment, mel_spec, mel_postnet_spec, stop_tokens, inputs = synthesis(model, text, TTS_CONFIG, use_cuda, ap, speaker_id, style_wav=None,
                                                                             truncated=False, enable_eos_bos_chars=TTS_CONFIG.enable_eos_bos_chars,
                                                                             backend='tflite')
    return mel_postnet_spec, TTS_CONFIG.audio['sample_rate']

## TTS Inference Helper

In [13]:
def run_tts_inference(text, model_name='Tacotron2', vocoder_name='MB-MelGAN', quantization='float16'):
    if model_name == 'Tacotron2':
        tac_output, sample_rate = run_tacotron2(text)
    elif model_name == 'FastSpeech2':
        _, tac_output = fastspeech_infer('fastspeech_quant.tflite', text)
        tac_output = np.squeeze(tac_output)
        sample_rate = 22050
    if vocoder_name == 'MelGAN':
        waveform = run_melgan(tac_output, quantization)
        waveform = np.squeeze(waveform)
    elif vocoder_name == 'MB-MelGAN':
        waveform = run_mb_melgan(tac_output.T)
        waveform = waveform[0, 0]
    elif vocoder_name == 'PWGAN':
        waveform = run_parallel_wavegan(tac_output, quantization)
        waveform = waveform[0, :, 0]
      
    IPython.display.display(IPython.display.Audio(waveform, rate=sample_rate))
    

## Choose model

In [32]:
tts_model = 'FastSpeech2' #@param ["Tacotron2", "FastSpeech2", "Glow-TTS"]

vocoder_model = 'MB-MelGAN' #@param ["MelGAN", "MB-MelGAN", "PWGAN"]

quantization = 'float16' #@param ["dr", "float16"]

## Inference

In [42]:
import re

text =  "Hello, Alice got in the habit of asking himself. Hello, Bill got in the habit of asking himself."
splits = re.split('\.', text)

for split in splits:
    run_tts_inference(split, tts_model, vocoder_model, quantization)

## Benchmarks

### MB-MELGAN

- Inference Time : 1.9ms
- Memory FootPrint : 15MB
- Model Size : 9.7MB

### TACOTRON2

- Inference Time

- Memory FootPrint

- Model Size :  28.67MB

###  MelGAN

#### Dynamic Range Quantization

- Inference Time : 

- Memory FootPrint :

- Model Size : 17MB 

#### Float16 Quantization

- Inference Time : 

- Memory FootPrint :

- Model Size : 8.4MB

### Parallel WaveGAN

#### Dynamic Range Quantization

- Inference Time : 

- Memory FootPrint :

- Model Size : 5.7MB 

#### Float16 Quantization

- Inference Time : 

- Memory FootPrint :

- Model Size : 3.2MB